In [100]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout,GridspecLayout,interact, interact_manual,VBox,HBox,Button, ButtonStyle, Output, FileUpload
from IPython.display import display, clear_output
import os
import subprocess
import lxml
# need pandas version >= 1.3

## Choisir le workflow à lancer

**Attention : si atoz, veiller à bien mettre à jour xslt/tables_conversion.xsl si besoin**
**Attention : ne pas oublier de modifier le path de tables_conversion dans la feuille de style atoz**

In [ ]:
atoz_cols_to_remove = ['Edition','Editor', 'Illustrator', 'DOI', 'PeerReviewed','CustomCoverageBegin',
       'CustomCoverageEnd', 'CoverageStatement', 'Embargo', 'CustomEmbargo',
       'Description', 'Subject', 'PackageContentType',
       'CreateCustom', 'HideOnPublicationFinder', 'Delete',
       'OrderedThroughEBSCO', 'IsCustom', 'UserDefinedField1',
       'UserDefinedField2', 'UserDefinedField3', 'UserDefinedField4',
       'UserDefinedField5', 'PackageType', 'AllowEbscoToAddNewTitles',
       'Unnamed: 40']

workflow = widgets.Dropdown(
    value='atoz', 
    options=['atoz', 'cairn_qsj', 'cairn_titre_a_titre', 'cyberlibris', 'numilog'], 
    description='Wotkflow select',
    style={'description_width': 'initial'},
    layout = Layout(width='50%')
)
filename = widgets.Text( 
    description='File name in /source_files/ (ex : atoz_export.csv)',
    style={'description_width': 'initial'},
    layout = Layout(width='80%')
)
button = widgets.Button(description="Launch!")
output = widgets.Output()

display(workflow,filename,button, output)

def file_path(relative_path):
    folder = os.path.dirname(os.path.abspath("__file__"))
    path_parts = relative_path.split("/")
    new_path = os.path.join(folder, *path_parts)
    return new_path

def on_button_clicked(b):
    with output:
        clear_output()
        if workflow.value == "atoz":
            df = pd.read_csv("source_files/"+filename.value, sep=',', encoding='utf8')
            #clean
            df = df.drop(atoz_cols_to_remove, axis=1).fillna("").replace("&", "&amp;")
            # custom bsc cleaning
            df.drop(df[(df.PackageName == 'Business Source Complete') & ((df.ResourceType == 'Report') | (df.ResourceType == 'Book Series'))].index, inplace=True)
            df.to_xml(path_or_buffer="temporary_files/atoz_export.xml", root_name='Resources', row_name='Resource', encoding='utf-8', xml_declaration=True, pretty_print=True, parser='lxml')
            print(subprocess.run(["run_saxon.bat",file_path("xslt/atoztemp4primo.xsl"),file_path("temporary_files/atoz_export.xml"),file_path("result_files/atoz.xml")], shell=True, check=True, capture_output=True))
        if workflow.value == "cairn_titre_a_titre":
            print("Attention passer dans MArcEdit d'abord")

button.on_click(on_button_clicked) 